
# Rule‑Based Logic for Hospital Prioritisation

> GPT5 generated baseline to beat

This notebook demonstrates a transparent **if/then** policy engine for:
- Prioritising current and incoming patients
- Flagging early discharge candidates
- Summarising blockers and investment levers

In [8]:

import pandas as pd
import numpy as np
from pathlib import Path

data_dir = Path("../data")
current = pd.read_csv(data_dir / "patients_current.csv")
coming  = pd.read_csv(data_dir / "patients_coming.csv")
historic= pd.read_csv(data_dir / "patients_historic.csv")

for df in (current, coming, historic):
    df.columns = [c.strip() for c in df.columns]

print("Loaded:", len(current), "current;", len(coming), "coming;", len(historic), "historic")
current.head(3)

Loaded: 100 current; 100 coming; 800 historic


,Current,age,sex,Complexity,Acuity,Primary Diagnosis Summary,Speciality,Vitals Trend,Waiting Time (days),Time since Admission (days),nextAction,blocker,Discharge Dependence
0,SP0801,96,F,0.95,1,D50-D53,General Internal Medicine,Stable,11,1,Treatment,Theatre slot availability,Low
1,SP0802,69,F,0.51,3,H30-H36,Ophthalmology,Improving,15,0,Review,No Blocker,High
2,SP0803,73,F,0.93,4,M00-M25,Trauma and Orthopaedics,Improving,35,3,Discharge,Awaiting Social Care,High


In [3]:

# Expected LOS from historic (by Speciality & Acuity), with fallbacks
group_cols = ["Speciality", "Acuity"]
los = (historic.groupby(group_cols)["Length of Stay (days)"].mean()
       .reset_index().rename(columns={"Length of Stay (days)":"exp_los"}))
spec_means = historic.groupby("Speciality")["Length of Stay (days)"].mean().to_dict()
overall = historic["Length of Stay (days)"].mean()

def attach_exp_los(df):
    out = df.merge(los, on=group_cols, how="left")
    out["exp_los"] = out["exp_los"].fillna(out["Speciality"].map(spec_means))
    out["exp_los"] = out["exp_los"].fillna(overall)
    return out

current = attach_exp_los(current)
coming  = attach_exp_los(coming)
current["remaining_los"] = np.maximum(current["exp_los"] - current["Time since Admission (days)"], 0.0)
current.head(3)

,Current,age,sex,Complexity,Acuity,Primary Diagnosis Summary,Speciality,Vitals Trend,Waiting Time (days),Time since Admission (days),nextAction,blocker,Discharge Dependence,exp_los,remaining_los
0,SP0801,96,F,0.95,1,D50-D53,General Internal Medicine,Stable,11,1,Treatment,Theatre slot availability,Low,4.926316,3.926316
1,SP0802,69,F,0.51,3,H30-H36,Ophthalmology,Improving,15,0,Review,No Blocker,High,1.125000,1.125000
2,SP0803,73,F,0.93,4,M00-M25,Trauma and Orthopaedics,Improving,35,3,Discharge,Awaiting Social Care,High,2.833333,0.000000


## Policy knobs

In [4]:
policy = {
    "safety_first_acuity_min": 4,     # Acuity >= this is critical
    "early_dc_acuity_max": 2,         # Max acuity for early discharge
    "early_dc_elapsed_ratio": 0.8,    # >= 80% of exp LOS elapsed
    "allow_vitals": {"Improving","Stable"},
    "require_no_blocker_for_dc": True,
    "low_dependence_values": {"Low"},
    "wait_weight": 1.0,
    "acuity_weight": 1.2,
    "vitals_weight": 2.0,
    "coming_wait_weight": 2.0,
}

## Rules & scoring

In [7]:
def early_discharge_rule(row, p=policy):
    if row["Acuity"] > p["early_dc_acuity_max"]:
        return False
    if row["Vitals Trend"] not in p["allow_vitals"]:
        return False
    if p["require_no_blocker_for_dc"] and row["blocker"] != "No Blocker":
        return False
    if row["Discharge Dependence"] not in p["low_dependence_values"]:
        return False
    if row["Time since Admission (days)"] < p["early_dc_elapsed_ratio"] * row["exp_los"]:
        return False
    return True

vmap = {"Deteriorating": 2, "Stable": 0, "Improving": -1}
def priority_score_current(df, p=policy):
    return (
        - (6 - df["Acuity"]) * p["acuity_weight"] +
        df["Vitals Trend"].map(vmap).fillna(0) * p["vitals_weight"] +
        (df["Waiting Time (days)"] / max(1, df["Waiting Time (days)"].max())) * p["wait_weight"]
    )

def priority_score_coming(df, p=policy):
    return (
        - (6 - df["Acuity"]) * p["acuity_weight"] +
        (df["Waiting Time (days)"] / max(1, df["Waiting Time (days)"].max())) * p["coming_wait_weight"]
    )

current["is_early_dc"] = current.apply(early_discharge_rule, axis=1)
current["priority_score"] = priority_score_current(current)
coming["priority_score"] = priority_score_coming(coming)

early_dc = current[current["is_early_dc"]].sort_values(["remaining_los","Acuity"])
top_current = current.sort_values("priority_score", ascending=False).head(10)
top_coming  = coming.sort_values("priority_score",  ascending=False).head(10)

print("Early discharge candidates:", len(early_dc))
display(early_dc[["Current","Speciality","Acuity","Vitals Trend","blocker","Discharge Dependence","Time since Admission (days)","exp_los","remaining_los"]].head(15))
display(top_current[["Current","Speciality","Acuity","Vitals Trend","Waiting Time (days)","priority_score","remaining_los"]])
display(top_coming[["Coming","Speciality","Acuity","Waiting Time (days)","priority_score","exp_los"]])

Early discharge candidates: 2


,Current,Speciality,Acuity,Vitals Trend,blocker,Discharge Dependence,Time since Admission (days),exp_los,remaining_los
26,SP0827,Ophthalmology,1,Stable,No Blocker,Low,1,1.000000,0.0
80,SP0881,General Internal Medicine,2,Improving,No Blocker,Low,77,4.602151,0.0


,Current,Speciality,Acuity,Vitals Trend,Waiting Time (days),priority_score,remaining_los
48,SP0849,Geriatric Medicine,2,Deteriorating,21,-0.757143,3.545455
5,SP0806,General Internal Medicine,2,Deteriorating,19,-0.761224,3.602151
6,SP0807,Geriatric Medicine,5,Stable,26,-1.146939,3.166667
14,SP0815,General Internal Medicine,5,Stable,11,-1.177551,0.000000
96,SP0897,General Internal Medicine,5,Stable,10,-1.179592,3.572917
52,SP0853,Gastronenterology,5,Stable,7,-1.185714,1.571429
19,SP0820,Trauma and Orthopaedics,4,Stable,231,-1.928571,2.833333
40,SP0841,General Internal Medicine,4,Stable,22,-2.355102,3.341176
33,SP0834,Paediatrics,4,Stable,3,-2.393878,0.000000
88,SP0889,General Internal Medicine,5,Improving,377,-2.430612,3.572917


,Coming,Speciality,Acuity,Waiting Time (days),priority_score,exp_los
16,SP0917,Trauma and Orthopaedics,5,172,-0.813483,2.181818
19,SP0920,Geriatric Medicine,5,68,-1.047191,4.166667
72,SP0973,Geriatric Medicine,5,61,-1.062921,4.166667
28,SP0929,General Internal Medicine,5,59,-1.067416,3.572917
47,SP0948,Trauma and Orthopaedics,5,55,-1.076404,2.181818
20,SP0921,Gastronenterology,5,55,-1.076404,2.571429
64,SP0965,General Internal Medicine,5,45,-1.098876,3.572917
17,SP0918,Gastronenterology,5,41,-1.107865,2.571429
40,SP0941,Trauma and Orthopaedics,5,35,-1.121348,2.181818
34,SP0935,Gastronenterology,5,17,-1.161798,2.571429
